In [1]:
import pandas as pd
import json
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from transformers import BertTokenizer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import torch
from transformers import BartTokenizer, BartForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from transformers import BartForSequenceClassification, BartTokenizer, AdamW, get_scheduler
from datasets import load_metric, list_metrics
from sklearn.metrics import accuracy_score

2024-08-03 15:12:05.727830: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 15:12:05.727968: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 15:12:05.886578: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load your data
train = pd.read_csv('/kaggle/input/datasetscsv/train_data.csv')
val = pd.read_csv('/kaggle/input/datasetscsv/val_data.csv')
test = pd.read_csv('/kaggle/input/datasetscsv/test.csv')

# Ensure the columns are named correctly
train_inputs = train['text'].astype(str).tolist()
train_labels = train['labels'].astype(str).tolist()

val_inputs = val['text'].astype(str).tolist()
val_labels = val['labels'].astype(str).tolist()

test_inputs = test['text'].astype(str).tolist()

# Encode labels
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform([label.strip("[]'") for label in train_labels])
val_labels = label_encoder.transform([label.strip("[]'") for label in val_labels])

# Save label mapping for later use
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)

Label Mapping: {'multi': 0, 'passage': 1, 'phrase': 2}


In [3]:
model_name = 'facebook/bart-base'
tokenizer = BartTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_inputs, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_inputs, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_inputs, truncation=True, padding=True, max_length=512)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [4]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

In [5]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [6]:
model = BartForSequenceClassification.from_pretrained(model_name, num_labels=len(set(train_labels)))

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 10
num_training_steps = num_epochs * len(train_loader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Lay

In [7]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [8]:
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    print(f"Epoch: {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

Epoch: 1/10, Loss: 1.1224925518035889
Epoch: 2/10, Loss: 1.032193660736084
Epoch: 3/10, Loss: 0.9308426380157471
Epoch: 4/10, Loss: 1.1798183917999268
Epoch: 5/10, Loss: 0.014200051315128803
Epoch: 6/10, Loss: 0.029484545812010765
Epoch: 7/10, Loss: 0.026720760390162468
Epoch: 8/10, Loss: 0.04601813107728958
Epoch: 9/10, Loss: 0.0004217004752717912
Epoch: 10/10, Loss: 0.001737871440127492


In [9]:
# Switch to evaluation mode
model.eval()

# Prepare to collect predictions and references
predictions = []
references = []

# Create DataLoader for the validation set
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Perform evaluation
with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=-1)
        predictions.extend(preds.cpu().numpy())
        references.extend(batch['labels'].cpu().numpy())

# Decode integer labels back to original label strings
decoded_predictions = label_encoder.inverse_transform(predictions)
decoded_references = label_encoder.inverse_transform(references)

# Calculate evaluation metrics
accuracy = accuracy_score(references, predictions)
f1 = f1_score(references, predictions, average='weighted')
report = classification_report(references, predictions, target_names=label_encoder.classes_)

print(f"Validation Accuracy: {accuracy}")
print(f"Validation F1 Score: {f1}")
print("Classification Report:")
print(report)

Validation Accuracy: 0.665
Validation F1 Score: 0.6635237880292499
Classification Report:
              precision    recall  f1-score   support

       multi       0.68      0.55      0.61        84
     passage       0.66      0.70      0.68       154
      phrase       0.67      0.69      0.68       162

    accuracy                           0.67       400
   macro avg       0.67      0.65      0.65       400
weighted avg       0.67      0.67      0.66       400



In [10]:
# Save the trained model and tokenizer
model.save_pretrained("/kaggle/working/bart_clickbait_model")
tokenizer.save_pretrained("/kaggle/working/bart_clickbait_tokenizer")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('/kaggle/working/bart_clickbait_tokenizer/tokenizer_config.json',
 '/kaggle/working/bart_clickbait_tokenizer/special_tokens_map.json',
 '/kaggle/working/bart_clickbait_tokenizer/vocab.json',
 '/kaggle/working/bart_clickbait_tokenizer/merges.txt',
 '/kaggle/working/bart_clickbait_tokenizer/added_tokens.json')

In [12]:
# Load test data
#test = pd.read_csv('/kaggle/working/test.csv')
test_inputs = test['text'].astype(str).tolist()

# Tokenize test data
test_encodings = tokenizer(test_inputs, truncation=True, padding=True, max_length=512)

# Define TestDataset class
class TestDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

# Create DataLoader for test data
test_dataset = TestDataset(test_encodings)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Switch to evaluation mode
model.eval()

# Prepare to collect predictions
predictions = []

# Perform prediction
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=-1)
        predictions.extend(preds.cpu().numpy())

# Decode integer labels back to original label strings
decoded_predictions = label_encoder.inverse_transform(predictions)

# Prepare the output DataFrame
pred_df = pd.DataFrame({
    'id': test['id'],  # Adjust based on your actual test data format
    'spoilerType': decoded_predictions
})

# Save predictions to CSV
pred_df.to_csv('/kaggle/working/output_predictions.csv', index=False)